In [1]:
# Parameters
cell_type = "DC"


In [2]:
import sys
import os

import anndata as ad

import pandas as pd
import numpy as np

from pyprojroot import here

import re

from tqdm import tqdm

In [3]:
def load_sorted_shap_values_fnames(
    cell_type: str = '',
    run_name: str = 'run1'):

    dirpath = here(f"inflammabucket_bkp/03_downstream_analysis/05_SHAP/results/04_shap/shap_vals")
    fname_regex = f'^{run_name}_{cell_type}_shap_values_' + r'(\d+)'
    results_batches = [
        (fname, int(re.search(fname_regex, fname).group(1)))
        for fname in os.listdir(dirpath)
        if re.search(fname_regex, fname)]

    sorted_files = sorted(results_batches, key=lambda x: x[1])

    sorted_filenames = [os.path.join(dirpath, filename) for filename, _ in sorted_files]

    return sorted_filenames

In [4]:
shap_val_list = [np.load(fname)['shap_values'] for fname in tqdm(load_sorted_shap_values_fnames(cell_type))] 


  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:04<00:16,  4.10s/it]

 40%|████      | 2/5 [00:08<00:12,  4.31s/it]

 60%|██████    | 3/5 [00:13<00:09,  4.81s/it]

 80%|████████  | 4/5 [00:20<00:05,  5.31s/it]

100%|██████████| 5/5 [00:23<00:00,  4.67s/it]

100%|██████████| 5/5 [00:23<00:00,  4.71s/it]

In [5]:
adata = ad.read_h5ad(here(f'inflammabucket_bkp/03_downstream_analysis/05_SHAP/data/{cell_type}_adataMerged_SPECTRAgenes.log1p.h5ad'), backed='r')

In [6]:
## Gene symbols and Disease labels
symbols_df = pd.read_pickle(here('inflammabucket_bkp/03_downstream_analysis/04_selected_gene_list.pkl'))
symbols_sorted = symbols_df.loc[adata.var_names].symbol.values

DISEASES = ['BRCA', 'CD', 'COPD', 'COVID', 'CRC', 'HBV', 'HIV', 'HNSCC', 'MS', 'NPC', 'PS', 'PSA', 'RA', 'SLE', 'UC', 'asthma', 'cirrhosis', 'flu', 'healthy', 'sepsis']

diseaseDict = dict()
for d in DISEASES:
    diseaseDict[d] = []

In [7]:
shape_values_matrix = np.concatenate(shap_val_list)

In [8]:
for idx, values in tqdm(adata.obs.groupby('sampleID', observed=True).indices.items()):
    geneXdisease_sample_i = pd.DataFrame(shape_values_matrix[values].mean(0))
    geneXdisease_sample_i.columns = DISEASES
    geneXdisease_sample_i.index = symbols_sorted
    for d in geneXdisease_sample_i.columns:
        diseaseDict[d].append(pd.DataFrame.from_dict({idx:geneXdisease_sample_i[d]}))

  0%|          | 0/801 [00:00<?, ?it/s]

  4%|▍         | 34/801 [00:00<00:02, 339.44it/s]

 10%|▉         | 79/801 [00:00<00:01, 404.20it/s]

 16%|█▌        | 127/801 [00:00<00:01, 436.29it/s]

 21%|██▏       | 171/801 [00:00<00:01, 343.47it/s]

...

 84%|████████▍ | 676/801 [00:01<00:00, 280.24it/s]

 89%|████████▉ | 714/801 [00:02<00:00, 300.66it/s]

 94%|█████████▍| 751/801 [00:02<00:00, 317.54it/s]

 99%|█████████▉| 793/801 [00:02<00:00, 343.39it/s]

100%|██████████| 801/801 [00:02<00:00, 350.84it/s]

In [9]:
for d in tqdm(DISEASES):
    pd.concat(diseaseDict[d], axis=1).to_csv(here(f'03_downstream_analysis/08_gene_importance/new_shap_plots/results/SHAP_AVGsamples/SHAP_AVGsample_{cell_type}_{d}.csv'))

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:00<00:13,  1.44it/s]

 10%|█         | 2/20 [00:01<00:18,  1.05s/it]

 15%|█▌        | 3/20 [00:02<00:17,  1.00s/it]

 20%|██        | 4/20 [00:04<00:18,  1.15s/it]

...

 85%|████████▌ | 17/20 [00:16<00:03,  1.02s/it]

 90%|█████████ | 18/20 [00:18<00:02,  1.12s/it]

 95%|█████████▌| 19/20 [00:19<00:01,  1.12s/it]

100%|██████████| 20/20 [00:20<00:00,  1.08s/it]

100%|██████████| 20/20 [00:20<00:00,  1.01s/it]